In [2]:
from bitstring import Bits
from enum import Enum
from functools import reduce

In [99]:
class BitTypes(Enum):
    Unsigned = 1
    SignedMagnitude = 2
    OnesComplement = 3
    TwosComplement = 4

class BitString():
    def __init__(self, bits, type_):
        self.bits = bits
        self.value = None
        self.magnitude = None
        self.mag_pad = ''
        self.ensure_length()
        self.set_type(type_)
    
    def set_type(self, type_):
        if type_ == BitTypes.Unsigned:
            # it is the literal value of the string
            self.magnitude = self.bits
            self._set_value()
        elif type_ == BitTypes.SignedMagnitude:
            self.mag_pad = ' '
            # it is signed, so it is the literal value of all bits following the sign bit
            self.magnitude = self.bits[1::]
            self._set_value()
            # if the sign bit is one, it's negative
            self.set_negative_value()
        elif type_ == BitTypes.OnesComplement:
            self.mag_pad = ' '
            if self.check_negative():
                self.flip()
                self.magnitude = self.magnitude[1::]
            else:
                self.magnitude = self.bits[1::]
            self._set_value()
            self.set_negative_value()
        elif type_ == BitTypes.TwosComplement:
            self.mag_pad = ' '
            if self.check_negative():
                self.flip()
                self.magnitude = self.magnitude[1::]
            else:
                self.magnitude = self.bits[1::]
            self.add('1')
            self._set_value()
            self.set_negative_value()
            
        else:
            raise KeyError('No valid BitType given')
        self.type_ = type_
            
    
    def ensure_length(self):
        if len(self.bits) == 1:
            self.bits = '0' + self.bits
        elif len(self.bits) == 0:
            self.bits = '00'
            
    def _set_value(self):
        self.value = int(self.magnitude, 2)
        
    def flip(self):
        self.magnitude = flip_bits(self.bits)
    
    def set_negative_value(self):
        self.value = negative_if_applicable(self.bits, self.value)
        
    def check_negative(self):
        return self.bits[0] == '1'
    
    def __str__(self):
        return '\n'.join([
            f'{self.type_.name} bitstring ::',
            f'  bits      :: {self.bits}',
            f'  magnitude :: {self.mag_pad}{self.magnitude}',
            f'  value     :: {self.value}',
        ])
    
    def add(self, other):
        if isinstance(other, BitString):
            other = other.magnitude
        elif isinstance(other, str):
            pass
        else:
            print('Value of adding bits is invalid')
            raise KeyError('Invalid addition')
        result = bin_add(self.magnitude, other)
        self.magnitude = result
        self.bits = self.bits[0] + self.magnitude
        self._set_value()
        
        

class BitManipulator():
    def __init__(self, base, bits=None):
        self.base = base
        self.bit_list = []
        self.bit_keys = {}
        

def flip_bits(bits):
    return ''.join(map(lambda x: '0' if x == '1' else '1', bits))

def negative_if_applicable(bits, value):
    if bits[0] == '1':
        return value * -1
    else:
        return value

def pad_for_addition(add_to, target):
    diff = len(target.bits) - len(add_to.bits)
    pad = '0' * diff
    if target.type_.value > 2:
        pad = flip_bits(pad)
    return BitString(add_to.bits[0] + pad + add_to.bits[1::], add_to.type_)
    
def add(one, two):
    
        
    if one.type_ != two.type_:
        # determine which one needs to be cast up
        to_convert, stay = (one, two) if one.type_.value < two.type_.value else (two, one)
        # add an extra 0 to the front if dealing with unsigned vs signed
        if to_convert.type_ == BitTypes.Unsigned:
            to_convert = BitString('0' + to_convert.bits, BitTypes.Unsigned)
        # determine which one is longer
        if len(to_convert.bits) < len(stay.bits):
            # add padding to to_convert
            to_convert = pad_for_addition(to_convert, stay)
        elif len(stay.bits) < len(to_convert):
            # add padding to stay
            stay = pad_for_addition(stay, to_convert)
        # else they're the same length
    print('Adding operation')
    print(f'  {to_convert.bits}')
    print(f'+ {stay.bits}')
    print('-'*len(stay.bits))
    print(f'result: {one.value + two.value}')
    return one.value + two.value


def bin_add(x, y, verbose=False):
    
    if len(x) > len(y):
        y = '0' * (len(x) - len(y)) + y
    elif len(y) > len(x):
        x = '0' * (len(y) - len(x)) + x
    if verbose:
        print(f'  {x}')
        print(f'+ {y}')
        print('-'*(len(x) + 2))
    carry = 0
    bits = []
    pad = ' '
    for i, j in zip(x[::-1], y[::-1]):
        res = int(i) + int(j) + carry
        y = 1 if res > 1 else 0
        carry = 1 if res > 1 else 0
        res = 1 if res == 3 else (0 if res == 2 else res)
        bits.append(str(res))
    if carry != 0:
        pad = ''
        bits.append(str(carry))
    result = ''.join(bits[::-1])
    if verbose:
        print(f' {pad}{result}')
        print(f' {pad}{int(result, 2)} -> base 10')
    return result

_DIGIT_DICT = {
    '0': 0,
    '1': 1,
    '2': 2,
    '3': 3,
    '4': 4,
    '5': 5,
    '6': 6,
    '7': 7,
    '8': 8,
    '9': 9,
    'a': 10,
    'b': 11,
    'c': 12,
    'd': 13,
    'e': 14,
    'f': 15,
    'g': 16,
    'h': 17,
    'i': 18,
    'j': 19,
    'k': 20,
    'l': 21,
    'm': 22,
    'n': 23,
    'o': 24,
    'p': 25,
    'q': 26,
    'r': 27,
    's': 28,
    't': 29,
    'u': 30,
    'v': 31,
    'w': 32,
    'x': 33,
    'y': 34,
    'z': 35,
}
_BASE_DICT = {
    '2': 2,
    '3': 3,
    '4': 4,
    '5': 5,
    '6': 6,
    '7': 7,
    '8': 8,
    '9': 9,
    '10': 10,
    '11': 11,
    '12': 12,
    '13': 13,
    '14': 14,
    '15': 15,
    '16': 16,
    '17': 17,
    '18': 18,
    '19': 19,
    '20': 20,
    '21': 21,
    '22': 22,
    '23': 23,
    '24': 24,
    '25': 25,
    '26': 26,
    '27': 27,
    '28': 28,
    '29': 29,
    '30': 30,
    '31': 31,
    '32': 32,
    '33': 33,
    '34': 34,
    '35': 35,
    '36': 36,
    'binary': 2,
    'bin': 2,
    'decimal': 10,
    'dec': 10,
    'hexadecimal': 16,
    'hex': 16,
}

def digit_values(digit, base):
    dval = _DIGIT_DICT[digit]
    if dval >= base:
        raise KeyError('Values larger than base detected')
    return dval

def convert_from_base_to_base10(value, base):
    base = _BASE_DICT[base]
    # for each individual digit in value (from right to left)
    # mutliply the value of the digit by base ^ position
    # sum all the values of the digits
    return reduce(lambda x, y: x + y, [digit_values(v, base) * (base**i) for i, v in enumerate(value[::-1])])


def convert_from_base10_to_base(value, base):
    dig_conv = {v: k for k, v in _DIGIT_DICT.items()}
    value = int(value)
    digits = []
    while value != 0:
        digits.append(dig_conv[value % base])
        value = int(value / base)
    return ''.join(digits[::-1])

def converter(value, base, targetbase, basetype=None):
    base_10 = None
    base = str(base).lower()
    targetbase = str(targetbase)
    # make sure base is supported
    if base not in _BASE_DICT.keys():
        print('Base not supported')
        return None
    # make sure target is supported
    if targetbase not in _BASE_DICT.keys():
        print('Base not supported')
        return None
    # convert value to base 10
    base_10 = convert_from_base_to_base10(value, base)
    print(f'value in base 10 is {base_10}')
    # convert value to targetbase
    value = convert_from_base10_to_base(base_10, _BASE_DICT[targetbase])
    print(f'value in base {targetbase} is {value}')
    return value

In [101]:
bs = BitString('101111101101', BitTypes.TwosComplement)
b2 = BitString('101111101101', BitTypes.OnesComplement)

In [98]:
converter('11001011101', 2, 16)

value in base 10 is 1629
value in base 16 is 65d


'65d'

In [86]:
convert_from_base_to_base10('taylorswift', '35')

80863990107065229

In [104]:
b3 = BitString('0000011010011', BitTypes.TwosComplement)

In [106]:
print(b3)

TwosComplement bitstring ::
  bits      :: 0000011010100
  magnitude ::  000011010100
  value     :: 212


In [46]:
converter('ffff', 16, 2)

value in base 10 is 65535
value in base 2 is 1111111111111111


'1111111111111111'